In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
work_dir="datasets/tcga/COAD/mirna_allsamples/"
#working_dir="../cancers/breast/"
os.chdir(work_dir)

# Regulatory network

In [ ]:
df_graph_meta = pd.read_csv("/home/jovyan/work/phd/regulatory/graph_metadata.csv", index_col=0)

In [ ]:
import networkx as nx
import sys
sys.path.append("/home/jovyan/work/phd/regulatory")
from regulatory import get_neighborns
from gseapy import enrichr

In [ ]:
gene_sets = ["/home/jovyan/work/phd/MSigDB/"+set for set in ['c1.all.v7.1.symbols.gmt',
                                             'c2.all.v7.1.symbols.gmt',
                                             #'c3.all.v7.1.symbols.gmt',
                                             'c4.all.v7.1.symbols.gmt',
                                             'c5.all.v7.1.symbols.gmt',
                                             'c6.all.v7.1.symbols.gmt',
                                             'c7.all.v7.1.symbols.gmt',
                                             'c8.all.v7.2.symbols.gmt',
                                             'h.all.v7.2.symbols.gmt',
                                                            ]
            ]

In [ ]:
g = nx.read_gpickle("/home/jovyan/work/phd/regulatory/graph.pkl.gz")

In [ ]:
df_topics = pd.read_csv("trisbm/trisbm_level_1_metadata.csv")
#df_key_dist = pd.read_csv("trisbm/trisbm_level_1_metadata.csv")
background = df_topics.values.ravel()
background = list(filter(lambda x: x!="nan", background.astype(str)))

res = {}

for metadatum in df_topics.columns:
    mirnas = df_topics[metadatum].dropna().values
    df_nodes = df_graph_meta[df_graph_meta["name"].isin(mirnas)]
    regulated = pd.DataFrame()
    for mirna in df_nodes["id"]:
        data = get_neighborns(mirna, g, df_graph_meta.set_index("id"))
        data = list(filter(lambda x: str(x)!="nan",data))
        regulated=regulated.join(
            pd.Series(index=data, 
                    name=mirna,
                    data = 1,
                    dtype=object),
            how="outer"
        )
    regulated = regulated.fillna(0).sum(1).sort_values(ascending=False)
    regulated = regulated[regulated>(len(mirnas)*0.05)] # genes regulated by at least 50% of the metadatum
    res[metadatum]=regulated.index
    #for gene in regulated.items():
    #    print(gene[0])
    try:
        print(enrichr(list(regulated.index), gene_sets).results.sort_values("Adjusted P-value", ascending=True)[["Term","Adjusted P-value"]])   
    except:
        pass
    print("\n\n**************************\n\n")

In [ ]:
max(map(len,res.values()))

In [ ]:
pd.DataFrame(data=[np.concatenate([li, np.repeat(np.nan,max(map(len,res.values()))-len(li))]) for li in res.values()]).transpose().dropna(how="all",axis=1).to_csv("metadata_regulated.csv")

In [ ]:
regulated